In [66]:
import pandas as pd
import numpy as np
import calendar as cal
import quandl, requests
from io import BytesIO
import os

In [4]:
file = "C:/Users/tomek/Desktop/CME_20181029.csv"

In [12]:
df = pd.read_csv(file,
                 error_bad_lines=False,
                 header=None,
                 parse_dates=[1],
                 names=['symbol', 'date', 'open', 'high', 'low', 'last', 'change', 'settle', 'volume', 'open_interest', 'x', 'y'])

In [13]:
df.count()

symbol           15121598
date             15121598
open              5725179
high              5827020
low               5827200
last               706823
change0           8626097
settle           15100578
volume           14976931
open_interest    14998092
x                    2066
y                   15410
dtype: int64

In [14]:
df.head()

,symbol,date,open,high,low,last,change0,settle,volume,open_interest,x,y
0,0DF2015,2014-02-18,NaN,NaN,NaN,NaN,4.596,578.571,0.0,10.0,NaN,NaN
1,0DF2015,2014-02-19,NaN,NaN,NaN,NaN,0.862,577.709,0.0,10.0,NaN,NaN
2,0DF2015,2014-02-20,NaN,NaN,NaN,NaN,0.925,578.634,0.0,10.0,NaN,NaN
3,0DF2015,2014-02-21,NaN,NaN,NaN,NaN,1.768,576.866,3.0,10.0,NaN,NaN
4,0DF2015,2014-02-24,NaN,NaN,NaN,NaN,3.261,580.127,0.0,13.0,NaN,NaN


In [22]:
df[df['symbol'] == 'ESZ2018'][df['date'] > '2018'].tail()

C:\Users\tomek\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,symbol,date,open,high,low,last,change0,settle,volume,open_interest,x,y
4726015,ESZ2018,2018-10-23,2755.75,2757.50,2692.25,2746.00,10.25,2746.25,2780303.0,2744048.0,NaN,NaN
4726016,ESZ2018,2018-10-24,2745.25,2750.00,2652.25,2674.75,82.00,2664.25,2783685.0,2762871.0,NaN,NaN
4726017,ESZ2018,2018-10-25,2673.75,2723.75,2658.75,2681.50,24.00,2688.25,2479997.0,2780617.0,NaN,NaN
4726018,ESZ2018,2018-10-26,2678.00,2692.75,2627.25,2662.25,18.75,2669.50,3410150.0,2777378.0,NaN,NaN
4726019,ESZ2018,2018-10-29,2667.00,2707.00,2603.00,2639.50,26.00,2643.50,2720779.0,2810469.0,NaN,NaN


In [29]:
df[~df['x'].isnull()].head()

,symbol,date,open,high,low,last,change0,settle,volume,open_interest,x,y


In [23]:
df['date'].min()

Timestamp('1959-07-01 00:00:00')

In [24]:
df['date'].max()

Timestamp('2018-10-29 00:00:00')

In [27]:
# drop option codes which are mistakenly included in Quandl file
df.drop(df[df['symbol'].str.len() > 8].index, inplace=True)
# drop various indexes included in Quandl file
df.drop(df[df['symbol'].str.contains('INDEX', regex=False) == True].index, inplace=True)

In [30]:
del df['x']
del df['y']

In [28]:
df[df['symbol'].str.contains('INDEX', regex=False) == True]

,symbol,date,open,high,low,last,change0,settle,volume,open_interest,x,y


In [31]:
df.head()

,symbol,date,open,high,low,last,change0,settle,volume,open_interest
0,0DF2015,2014-02-18,NaN,NaN,NaN,NaN,4.596,578.571,0.0,10.0
1,0DF2015,2014-02-19,NaN,NaN,NaN,NaN,0.862,577.709,0.0,10.0
2,0DF2015,2014-02-20,NaN,NaN,NaN,NaN,0.925,578.634,0.0,10.0
3,0DF2015,2014-02-21,NaN,NaN,NaN,NaN,1.768,576.866,3.0,10.0
4,0DF2015,2014-02-24,NaN,NaN,NaN,NaN,3.261,580.127,0.0,13.0


In [33]:
df[df['symbol'] == 'CBV2020']

,symbol,date,open,high,low,last,change0,settle,volume,open_interest


In [37]:
EXPIRATION_DATES = 'C:/Users/tomek/zipline/expiration_dates.csv'
# lookup table used to get contract expiration dates
expiration_table = pd.read_csv(EXPIRATION_DATES,
                               usecols=['symbol', 'expiration_date'],
                               parse_dates=['expiration_date'], 
                              index_col='symbol')

In [40]:
def third_friday(symbol):
    """
    Return third Friday of the expiration month for the passed symbol.
    Used as a fallback if real expiration date cannot be found.
    """
    year = int(symbol[-4:])
    month = int(CME_CODE_TO_MONTH[symbol[-5]])
    c = cal.Calendar(firstweekday=cal.SATURDAY)
    day = c.monthdatescalendar(year, month)[2][-1]
    return pd.to_datetime(day)


In [41]:
def get_expiration(symbol):
    """
    Return expiration date from csv file. If the date is not available, fall back on using 
    third Friday of the expiration month.
    """
    try:
        return pd.to_datetime(expiration_table.loc[symbol,'expiration_date'])
    except KeyError:
        #log.error('key value: {}'.format(symbol))
        return third_friday(symbol)

In [45]:
CME_CODE_TO_MONTH = dict(zip('FGHJKMNQUVXZ', range(1, 13)))

In [60]:
META_FILE = 'C:/Users/tomek/zipline/meta.csv'     
def get_meta_df():
    """
    Fetch metadata from csv file based on modified quandl supplied meta file.

    """
    return pd.read_csv(META_FILE, usecols=['root_symbol', 'name', 'exchange', 'multiplier',
                                         'tick_size', 'sector', 'sub_sector',])


In [58]:
data = df.groupby(
        by='symbol'
    ).agg(
        {'date': [np.min, np.max]}
    )

data.reset_index(inplace=True)
data.columns = data.columns.get_level_values(1)
data.rename(columns={'': 'symbol', 'amin':'start_date', 'amax': 'end_date'}, inplace=True)  
data['first_traded'] = data['start_date']
# extract expiration dates
d = data['end_date'].max() - pd.Timedelta(days=2)
data['active'] = data['end_date'] >= d
# expired contracts have data up to expiration date
data['expiration_date'] = data[data['active']].symbol.apply(lambda x: get_expiration(x)).combine_first(
        data[~data['active']]['end_date'])
data['expiration_date'] = data['expiration_date'].astype('datetime64[ns]')
data['auto_close_date'] = data['expiration_date'] - pd.Timedelta(days=2)
data['notice_date'] = data['auto_close_date'] #- pd.Timedelta(days=1)
data['expiration_year'] = data.symbol.apply(lambda x: x[-4:])

In [61]:
meta = get_meta_df()

In [62]:
meta.head()

,root_symbol,exchange,name,multiplier,tick_size,sector,sub_sector
0,0D,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.00100,Energy,Refined Products
1,0F,NYMEX,Mini Singapore Fuel Oil 180 cst (Platts) Futures,100.0,0.00100,Energy,Refined Products
2,1N,NYMEX,Singapore Mogas 92 Unleaded (Platts) Futures,1000.0,0.00100,Energy,Refined Products
3,1NB,NYMEX,Singapore Mogas 92 Unleaded (Platts) Brent Cra...,1000.0,0.00100,Energy,Refined Products
4,1R,NYMEX,Propane Non-LDH Mont Belvieu (OPIS) Futures,42000.0,0.00001,Energy,Petrochemicals


In [63]:
data.head()

,symbol,start_date,end_date,first_traded,active,expiration_date,auto_close_date,notice_date,expiration_year
0,0DF2015,2014-02-18,2016-12-28,2014-02-18,False,2016-12-28,2016-12-26,2016-12-26,2015
1,0DF2016,2014-02-18,2016-02-02,2014-02-18,False,2016-02-02,2016-01-31,2016-01-31,2016
2,0DF2017,2014-11-21,2017-02-02,2014-11-21,False,2017-02-02,2017-01-31,2017-01-31,2017
3,0DF2018,2014-11-21,2018-02-02,2014-11-21,False,2018-02-02,2018-01-31,2018-01-31,2018
4,0DF2019,2015-02-12,2018-10-29,2015-02-12,True,2019-01-31,2019-01-29,2019-01-29,2019


In [67]:
api_key = os.environ.get('QUANDL_API_KEY')

In [68]:
def fetch_quandl_specs_table(api_key, show_progress=False):
    """
    Return quandl spec file with a list of all available contracts.
    """
    if show_progress:
        log.info('Downloading metadata file from quandl')

    r = requests.get('https://www.quandl.com/api/v3/databases/CME/metadata?api_key={}'.format(api_key))
    r.raise_for_status()
    return pd.read_csv(BytesIO(r.content), compression='zip',
                       parse_dates=['from_date', 'to_date'])


In [69]:
quandl_specs = fetch_quandl_specs_table(api_key)

In [74]:
quandl_specs.head()

,code,name,description,refreshed_at,from_date,to_date
0,0DF2015,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-12-28
1,0DF2016,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-02-02
2,0DF2017,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2017-02-03 01:05:31,2014-11-21,2017-02-02
3,0DF2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-02-03 01:07:36,2014-11-21,2018-02-02
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-10-31 00:11:23,2015-02-12,2018-10-30


In [80]:
quandl_specs['name']

0         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
1         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
2         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
3         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
4         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
5         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
6         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
7         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
8         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
9         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
10        Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
11        Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
12        Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
13        Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
14        Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
15        Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
16        Mini European 3.5% Fuel Oil Ba

In [90]:
a = quandl_specs['name']
a.index = quandl_specs['code']
a

code
0DF2015         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DF2016         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DF2017         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DF2018         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DF2019         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DF2020         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DF2021         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DG2014         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DG2015         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DG2016         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DG2017         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DG2018         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DG2019         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DG2020         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DG2021         Mini European 3.5% Fuel Oil Barges FOB Rdam (P...
0DH20

In [70]:
data['root_symbol'] = [s[:-5] for s in data.symbol.unique() ]
names = pd.Series(quandl_specs['name'], index=quandl_specs['code'])
data['asset_name'] = data.symbol.apply(lambda x: names.loc[x])

data = data.merge(meta, on='root_symbol', how='left')
# precede single character roots with _, eg. C (corn) becomes _C
data['root_symbol'] = data['root_symbol'].apply(lambda x: '_' + x if len(x) < 2 else x)

In [73]:
names

code
0DF2015         NaN
0DF2016         NaN
0DF2017         NaN
0DF2018         NaN
0DF2019         NaN
0DF2020         NaN
0DF2021         NaN
0DG2014         NaN
0DG2015         NaN
0DG2016         NaN
0DG2017         NaN
0DG2018         NaN
0DG2019         NaN
0DG2020         NaN
0DG2021         NaN
0DH2014         NaN
0DH2015         NaN
0DH2016         NaN
0DH2017         NaN
0DH2018         NaN
0DH2019         NaN
0DH2020         NaN
0DH2021         NaN
0DJ2014         NaN
0DJ2015         NaN
0DJ2016         NaN
0DJ2017         NaN
0DJ2018         NaN
0DJ2019         NaN
0DJ2020         NaN
               ... 
ZRG2017C4_25    NaN
ZRG2017C5_25    NaN
ZRG2018C3_75    NaN
ZRG2018C3_9     NaN
ZRG2018C4       NaN
ZRG2018C4_25    NaN
ZRG2018C4_5     NaN
ZRZ2016C3_5     NaN
ZRZ2016C3_75    NaN
ZRZ2016C3_9     NaN
ZRZ2016C4       NaN
ZRZ2016C4_25    NaN
ZRZ2016C5_25    NaN
ZRZ2017C3_75    NaN
ZRZ2017C3_9     NaN
ZRZ2017C4       NaN
ZRZ2017C4_25    NaN
ZRZ2017C4_5     NaN
ZU2014         

In [71]:
data.head()

,symbol,start_date,end_date,first_traded,active,expiration_date,auto_close_date,notice_date,expiration_year,root_symbol,asset_name,exchange,name,multiplier,tick_size,sector,sub_sector
0,0DF2015,2014-02-18,2016-12-28,2014-02-18,False,2016-12-28,2016-12-26,2016-12-26,2015,0D,NaN,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.001,Energy,Refined Products
1,0DF2016,2014-02-18,2016-02-02,2014-02-18,False,2016-02-02,2016-01-31,2016-01-31,2016,0D,NaN,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.001,Energy,Refined Products
2,0DF2017,2014-11-21,2017-02-02,2014-11-21,False,2017-02-02,2017-01-31,2017-01-31,2017,0D,NaN,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.001,Energy,Refined Products
3,0DF2018,2014-11-21,2018-02-02,2014-11-21,False,2018-02-02,2018-01-31,2018-01-31,2018,0D,NaN,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.001,Energy,Refined Products
4,0DF2019,2015-02-12,2018-10-29,2015-02-12,True,2019-01-31,2019-01-29,2019-01-29,2019,0D,NaN,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.001,Energy,Refined Products


In [72]:
data.columns

Index(['symbol', 'start_date', 'end_date', 'first_traded', 'active',
       'expiration_date', 'auto_close_date', 'notice_date', 'expiration_year',
       'root_symbol', 'asset_name', 'exchange', 'name', 'multiplier',
       'tick_size', 'sector', 'sub_sector'],
      dtype='object')

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38374 entries, 0 to 38373
Data columns (total 9 columns):
symbol             38374 non-null object
start_date         38374 non-null datetime64[ns]
end_date           38374 non-null datetime64[ns]
first_traded       38374 non-null datetime64[ns]
active             38374 non-null bool
expiration_date    38374 non-null datetime64[ns]
auto_close_date    38374 non-null datetime64[ns]
notice_date        38374 non-null datetime64[ns]
expiration_year    38374 non-null object
dtypes: bool(1), datetime64[ns](6), object(2)
memory usage: 2.4+ MB
